# <font color='blue'>Projeto de Análise de Dados</font>
### <font color='blue'>Responder 5 perguntas de negócio</font> 

## <font color='blue'>Problema de negócio (Objetivo)</font> 

Nosso objetivo aqui é responder 5 perguntas de negócio sobre dados de ataques contemporâneos sintéticos

Perguntas que serão respondidas:

    1-Quantas vezes ocorre o ataque (attack_cat) do tipo Exploits?
	2-Para o ataque do tipo DoS quantas vezes o serviço (service) era http?
	3-Para o ataque do tipo Schellcode qual foi o protocolo (proto) mais comum?
	4-Qual tipo de ataque teve a maior média de duração (dur) do ataque?
	5-Para o tipo de ataque Backdoor e service ftp, qual foi a média de duração?


## <font color='blue'>Carregamento dos dados</font> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
# Importação do arquivo CSV
df=pd.read_csv(r'UNSW_NB15_training.csv')

## <font color='blue'>Análise Exploratória inicial</font> 

In [3]:
# Visualizando nosso Dataframe
df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.090200,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.000300,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.005100,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.660800,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.002500,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69598,69995,0.464833,tcp,-,CON,6,2,1012,86,15.059172,...,1,5,0,0,0,2,5,0,Normal,0
69599,69996,1.215478,tcp,-,FIN,60,14,68201,612,60.058679,...,1,1,0,0,1,2,1,0,Normal,0
69600,69997,0.635716,tcp,-,CON,6,2,1012,86,11.011206,...,1,2,0,0,0,2,2,0,Normal,0
69601,69998,0.749725,tcp,http,FIN,60,14,68199,612,97.369037,...,1,1,0,0,1,2,1,0,Normal,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69603 entries, 0 to 69602
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 69603 non-null  int64  
 1   dur                69603 non-null  float64
 2   proto              69603 non-null  object 
 3   service            69603 non-null  object 
 4   state              69603 non-null  object 
 5   spkts              69603 non-null  int64  
 6   dpkts              69603 non-null  int64  
 7   sbytes             69603 non-null  int64  
 8   dbytes             69603 non-null  int64  
 9   rate               69603 non-null  float64
 10  sttl               69603 non-null  int64  
 11  dttl               69603 non-null  int64  
 12  sload              69603 non-null  float64
 13  dload              69603 non-null  float64
 14  sloss              69603 non-null  int64  
 15  dloss              69603 non-null  int64  
 16  sinpkt             696

In [5]:
df.columns

Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')

Analisando nosso conjunto de dados e o objetivo, vimos que poucas dessas colunas vão ser realmente necessário para a resolução do problema.

As colunas que usaremos para resolver o problema será:

`attack_cat`,`dur`,`proto` e `service`

Todas as outras colunas poderão ser deletadas!

Vimos também que a maioria das perguntas podem ser respondidas com um simples agrupamento e uma filtragem

In [6]:
# Pegando os valores distintos das colunas que usaremos 
# Verificando valores únicos
lista=['attack_cat','dur','proto','service']
for col in lista:
    
    # Obtém uma lista de valores únicos
    list_of_unique_values = df[col].unique()
    
    # Se o número de valores exclusivos for menor que 15, imprima os valores. 
    # Caso contrário, imprima o número de valores exclusivos
    if len(list_of_unique_values) < 15:
        print("\n")
        print(col + ': ' + str(len(list_of_unique_values)) + ' valores únicos')
        print(list_of_unique_values)
    else:
        print("\n")
        print(col + ': ' + str(len(list_of_unique_values)) + ' valores únicos')



attack_cat: 10 valores únicos
['Normal' 'Reconnaissance' 'Backdoor' 'DoS' 'Exploits' 'Analysis'
 'Fuzzers' 'Worms' 'Shellcode' 'Generic']


dur: 31328 valores únicos


proto: 131 valores únicos


service: 13 valores únicos
['-' 'http' 'ftp' 'ftp-data' 'smtp' 'pop3' 'dns' 'snmp' 'ssl' 'dhcp' 'irc'
 'radius' 'ssh']


## <font color='blue'>Limpeza dos dados</font> 

Vamos preparar o nosso conjunto de dados, realizando limpeza neste

#### <font color='blue'>Remoção de colunas</font> 

In [7]:
# Criando uma copia do df
df_copia=df.copy()

In [8]:
# Criando uma lista somente das colunas que iremos usar
lista=['attack_cat','dur','proto','service']

In [9]:
# Pecorrendo nossa tabela
lista_remover=[]
for i in df.columns:
    if i not in lista:
        lista_remover.append(i)
# Deletar as colunas da lista de colunas que não iremos usar 
df_copia=df_copia.drop(lista_remover,axis=1)

#### <font color='blue'>Tratamento de valores ausentes</font> 

Na análise exploratória inicial vimos que não tem nenhum valor ausente não nulo que possa comprometer nosso conjunto de dados

Mas vamos analisar para vermos se tem um ausente nulo

In [10]:
# Verifica a quantidade de Valores ausentes em todo meu dataframe
def func_calc_percentual_valores_ausentes(df):
    totalCells = np.product(df.shape)
    missingCount = df.isnull().sum()
    totalMissing = missingCount.sum()
    print("O dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

In [11]:
# Executa função
func_calc_percentual_valores_ausentes(df_copia)

O dataset tem 0.0 % de valores ausentes.


In [12]:
# Calculando o total de valores ausentes e o percentual por variável e colocando em um Dataset
# Função para calcular valores ausentes por coluna
def func_calc_percentual_valores_ausentes_coluna(df_copia):
    # Total de valores ausentes
    mis_val = df.isnull().sum()
    # Porcentagem de valores ausentes
    mis_val_percent = 100 * mis_val / len(df)
    # Tipo de dado das colunas com valores ausentes
    mis_val_dtype = df.dtypes
    # Cria uma tabela com os resultados
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)
    # Renomear as colunas
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Valores Ausentes', 1 : '% de Valores Ausentes', 2: 'Dtype'})
    # Classifica a tabela por porcentagem de valores ausentes de forma decrescente e remove colunas sem valores faltantes
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)
    # Print 
    print ("O dataset tem " + str(df.shape[1]) + " colunas.\n"
        "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")
    if mis_val_table_ren_columns.shape[0] == 0:
        return
    # Retorna o dataframe com informações ausentes
    return mis_val_table_ren_columns

In [13]:
func_calc_percentual_valores_ausentes(df_copia)
df_missing = func_calc_percentual_valores_ausentes_coluna(df_copia)
# Visualizando o Dataframe
df_missing

O dataset tem 0.0 % de valores ausentes.
O dataset tem 45 colunas.
Encontrado: 0 colunas que têm valores ausentes.


Analisando nosso conjunto de dados, vimos que não há nenhum valor ausente no nossos dados

Então vamos seguir em frente

In [14]:
# Avaliação
print(df_copia.info())
print('--------------------------------------------')
print(df_copia.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69603 entries, 0 to 69602
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dur         69603 non-null  float64
 1   proto       69603 non-null  object 
 2   service     69603 non-null  object 
 3   attack_cat  69603 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.1+ MB
None
--------------------------------------------
(69603, 4)


## <font color='blue'>Análise Exploratória parte 2</font> 

Nesta análise exploratória, vamos trabalhar para responder as perguntas de negócio

In [15]:
df_copia.head(10)

,dur,proto,service,attack_cat
0,0.000011,udp,-,Normal
1,0.000008,udp,-,Normal
2,0.000005,udp,-,Normal
3,0.000006,udp,-,Normal
4,0.000010,udp,-,Normal
5,0.000003,udp,-,Normal
6,0.000006,udp,-,Normal
7,0.000028,udp,-,Normal
8,0.000000,arp,-,Normal
9,0.000000,arp,-,Normal


# Perguntas que serão respondidas :

    1-Quantas vezes ocorre o ataque (attack_cat) do tipo Exploits?
	2-Para o ataque do tipo DoS quantas vezes o serviço (service) era http?
	3-Para o ataque do tipo Schellcode qual foi o protocolo (proto) mais comum?
	4-Qual tipo de ataque teve a maior média de duração (dur) do ataque?
	5-Para o tipo de ataque Backdoor e service ftp, qual foi a média de duração?

## 1-Quantas vezes ocorre o ataque (attack_cat) do tipo Exploits?

In [16]:
# Realizando filtro no nosso dataframe
tabela1=df_copia[df_copia['attack_cat']=='Exploits']
# Avaliação
tabela1

,dur,proto,service,attack_cat
247,0.000009,sctp,-,Exploits
249,0.000009,gre,-,Exploits
250,0.000009,gre,-,Exploits
251,0.000009,gre,-,Exploits
252,0.000016,udp,-,Exploits
...,...,...,...,...
65441,1.130733,tcp,-,Exploits
65442,1.130733,tcp,-,Exploits
65443,0.207156,tcp,-,Exploits
65444,0.205856,tcp,-,Exploits


In [27]:
# Quantidade de ataques
print(tabela1['attack_cat'].count())

10999


# 2-Para o ataque do tipo DoS quantas vezes o serviço (service) era http?

In [18]:
# Realizando filtro no nosso dataframe
tabela2=df_copia[df_copia['attack_cat']=='DoS']
# Realizando filtro na nossa tabela
tabela2=tabela2[tabela2['service']=='http']
# Avaliação
tabela2

,dur,proto,service,attack_cat
336,1.438237,tcp,http,DoS
338,1.465404,tcp,http,DoS
415,0.417471,tcp,http,DoS
422,0.338931,tcp,http,DoS
429,0.340732,tcp,http,DoS
...,...,...,...,...
59532,0.183187,tcp,http,DoS
59537,0.894889,tcp,http,DoS
61489,4.437746,tcp,http,DoS
62780,0.346624,tcp,http,DoS


In [28]:
# Quantidade de ataques
print(tabela2['service'].count())

484


# 3-Para o ataque do tipo Shellcode qual foi o protocolo (proto) mais comum?

In [20]:
# Realizando filtro no nosso dataframe
tabela3=df_copia[df_copia['attack_cat']=='Shellcode']
# Avaliação 
tabela3

,dur,proto,service,attack_cat
425,0.320574,tcp,-,Shellcode
432,0.000007,udp,-,Shellcode
441,0.521790,tcp,-,Shellcode
447,0.376810,tcp,-,Shellcode
577,0.000010,udp,-,Shellcode
...,...,...,...,...
65083,1.036067,tcp,-,Shellcode
65159,0.491988,tcp,-,Shellcode
65190,0.706841,tcp,-,Shellcode
65212,0.655510,tcp,-,Shellcode


In [21]:
# Quantidade por protocolo
tabela3['proto'].value_counts()

tcp    190
udp    180
Name: proto, dtype: int64

# 4-Qual tipo de ataque teve a maior média de duração (dur) do ataque?

In [22]:
# Realizando agrupamento por ataque
tabela4=df_copia[['dur','attack_cat']].groupby('attack_cat').mean()
# Realizando a ordenação por ordem crescente
tabela4=tabela4.sort_values(by='dur',ascending=False)
# Avaliação
tabela4

,dur
attack_cat,
Fuzzers,2.143183
DoS,2.075084
Exploits,1.717414
Worms,1.061996
Normal,0.983500
Backdoor,0.923928
Reconnaissance,0.843122
Shellcode,0.353703
Analysis,0.280153


In [31]:
# Pegando a maior média
print(tabela4['dur'].max())

2.1431825994568907


# 5-Para o tipo de ataque Backdoor e service ftp, qual foi a média de duração?

In [24]:
# Realizando agrupamento por ataque
tabela5=df_copia[df_copia['attack_cat']=='Backdoor']
tabela5=tabela5[tabela5['service']=='ftp']
# Fazendo o agrupamento e agregação com media
grouped_df = tabela5.groupby(['service','attack_cat'])["dur"].aggregate("mean").reset_index()

In [25]:
# Pegando a média
media=grouped_df['dur'].values

In [26]:
print(f'A média de duração foi de {media}')

A média de duração foi de [0.727696]


## <font color='blue'>Relatório Final</font> 

Com base em minha análise temos o seguinte relatório:

1- O ataque " Exploits" ocorreu 10999 vezes

2- Para o ataque do tipo "DoS" tivemos 484 ataques do tipo "http"

3- Para o ataque do tipo "Schellcode" o protocolo mais comum foi o "tcp"

4- O ataque que deve a maior média de duração doi o "Fuzzers"

5- Para o ataque do tipo "Backdoor" e serviço "ftp" a média de duração doi de 0.727696


# <font color='blue'>FIM</font> 